In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
local_path = "files/test2.pdf"

loader = PyPDFLoader(local_path)
pages = loader.load_and_split()

In [3]:
pages[0].page_content

'Гра нты для молодых\nпре дпри нимателей\nБлагодаря господдержке граждане до 25 лет, которые решили открыть свое\nдело, могут получить грант от 100 до 500 тыс. рублей (или до 1 млн рублей в\nтом случае, если деятельность ведется в Арктической зоне). Средства могут\nполучить как индивидуальные предприниматели, так и учредители\nпредприятий.\xa0\nFAQ  по гра нтам для молодёжи\nКто может получить гранты?\nИП и юридические лица, основанные лицами в возрасте от 14 до 25 лет\n(включительн о). До 18 лет – с разрешения родителей. При этом если речь о\nюрлице, то молодой человек должен владеть долей в компании свыше 50% .\nКакова сумма гранта?\nМ инимальн ая – 100 тыс. рублей.\nМ аксимальн ая – 500 тыс. рублей (до 1 млн рублей для Арктической\nзоны: М урманская и Архангельская области, Ненецкий и Ямало-Ненецкий\nАО, Чукотка, Карелия, Коми, Якутия, Красноярский край).Ме ры поддержки Новости Кадры для экономики Центры «Мо й бизнес» База знаний Как открыть бизнес\n\uf189'

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [5]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)

In [6]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 5/5 [00:12<00:00,  2.60s/it]


In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [8]:
# LLM from Ollama
local_model = "llama3:8b"
llm = ChatOllama(model=local_model)

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Вы - ассистент по языковой модели искусственного интеллекта для ответа по законодательному документу.
 Не нужно придумывать своих условий или что-то подобное, только что представленно в тексте.
 Ваша задача - найти информацию из текста документа условия участия и только их.
 Никакой другой или вводной информации не надо. 
 Отвечай только на русском языке для этого.
 Исходный вопрос: {question}""",
)

In [10]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Отвечай на вопросы без вводной инфформации согласно контексту.  Только список. В конце переведи свой ответ на русский язык.:
{context}
Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
respoonse = chain.invoke("Какие условия участия в программе представленном в данном файле? В конце переведи свой ответ на русский")
respoonse

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


'Condition for receiving a grant:\n\n1. No debts over 1000 rubles.\n2. Completion of free training at the "My Business" center on the basics of entrepreneurial activity (you must understand the conditions of conducting business, measures to support it, and new developments in legislation). The duration of the training is not less than 16 hours.\n3. Co-financing of at least 25% of the project cost (for example, if you want to purchase equipment using grant funds, you must add 25% from your own funds; if you do not have your own funds, you can take a subsidized microloan from the "My Business" center - it is provided for up to 3 years with a maximum amount of 5 million rubles).\n\nTranslation:\n\nУсловия получения гранта:\n\n1. Нет долгов свыше 1000 рублей.\n2. Окончание бесплатного обучения в центре "Мой бизнес" по основам предпринимательской деятельности (должны разбираться в условиях ведения бизнеса, мерах поддержки и нововведениях в законодательстве). Длительность обучения - не менее

In [13]:
import torch
import nltk
from sacrebleu.metrics import BLEU

from rouge import Rouge
rouge_scorer = Rouge()

In [14]:
def evaluate_response(reference, response):
    # BLEU оценка
    bleu_scorer = BLEU(effective_order=True)
    blue_score = bleu_scorer.sentence_score(
        hypothesis=response,
        references=[reference],
    )

    # ROUGE оценка
    score = rouge_scorer.get_scores(
        hyps=response,
        refs=reference,
    )
    rouge_score = score[0]["rouge-l"]["f"]


    return blue_score, rouge_score

In [15]:
reference_text = '''Граждане до 25 лет, которые решили открыть своедело.Нет долгов по налогам/взносам больше 1000 рублей.
Прохождение бесплатного обучения в центре «Мой бизнес» по
основам предпринимательской деятельности (вы должны разбираться в
условиях ведения бизнеса, мерах поддержки и нововведениях в
законодательстве). Длительность обучения – не менее 16 часов. Если вы уже
обучились по программе центра или Корпорации МСП, то можете
использовать полученный сертификат (срок его действия не менее года).
Софинансирование не менее 25% от стоимости проекта (например,
если вы хотите приобрести на средства гранта оборудование, то 25% от его
стоимости должны добавить из своих денег; если своих средств еще нет, то
можно, к примеру, взять льготный микрозаём в центре «Мой бизнес» – он
предоставляется на срок до 3 лет в объеме до 5 млн рублей).
'''

In [16]:
bleu, rouge = evaluate_response(reference_text, respoonse)

In [17]:
print("BLEU:", bleu.score/100)
print("ROUGE:", rouge)

BLEU: 0.3089601478770658
ROUGE: 0.4899598346194417
